# 1. 임베딩

In [1]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("jinaai/jina-embeddings-v3", trust_remote_code=True)

sentences = [
    "The weather is lovely today.",
    "It's so sunny outside!",
    "He drove to the stadium."
]
embeddings = model.encode(sentences)

similarities = model.similarity(embeddings, embeddings)
print(similarities.shape)
# [3, 3]

README.md:   0%|          | 0.00/734k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/jinaai/jina-embeddings-v3:
- custom_st.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


torch.Size([3, 3])


In [2]:
import re
from langchain.document_loaders import PyMuPDFLoader

def clean_legal_text(text):
    # Remove page headers/footers (e.g., 법제처, 국가법령정보센터, page numbers)
    text = re.sub(r'^\s*법제처.*국가법령정보센터\s*$|^\s*\d+\s*$', '', text, flags=re.MULTILINE)
    
    # Remove contact information and department names
    text = re.sub(r'\(.*?\)\s*\d{2,3}-\d{3,4}-\d{4}', '', text)
    
    # Remove excessive newlines and spaces
    text = re.sub(r'(\n\s*){2,}', '\n', text)  # Collapse multiple newlines
    text = re.sub(r'\s{2,}', ' ', text)        # Collapse multiple spaces
    
    # Remove metadata like [시행 2024.10.20.]
    text = re.sub(r'^\s*\[.*?\]$', '', text, flags=re.MULTILINE)
    
    # Trim leading and trailing whitespace
    return text.strip()

def keyword_based_splitter(text, pattern=r'제\s*\d+\s*조\s*\(.*?\)'):
    # Find all matches for the pattern
    matches = re.finditer(pattern, text)

    # Extract start positions of each match
    split_positions = [match.start() for match in matches]

    # Add the end of the text as the last position
    split_positions.append(len(text))

    # Split the text based on the positions
    segments = []
    for i in range(len(split_positions) - 1):
        start = split_positions[i]
        end = split_positions[i + 1]
        segments.append(text[start:end].strip())
    
    return segments

def process_and_split_pdf(file_path, pattern=r'제\s*\d+\s*조\s*\(.*?\)'):
    # Load the PDF using PyMuPDFLoader
    loader = PyMuPDFLoader(file_path)
    documents = loader.load()

    # Extract and clean text from the documents
    cleaned_texts = []
    for doc in documents:
        cleaned_texts.append(clean_legal_text(doc.page_content))
    
    # Combine cleaned texts into one and split based on the pattern
    combined_cleaned_text = '\n'.join(cleaned_texts)
    split_segments = keyword_based_splitter(combined_cleaned_text, pattern)
    
    return split_segments

# Example usage
file_path = "file/도로 교통법/도로교통법 시행규칙(행정안전부령)(제00431호)(20241020).pdf"  # Replace with your file path
split_segments = process_and_split_pdf(file_path)

# Display split segments
for i, segment in enumerate(split_segments):
    print(f"Segment {i + 1}:\n{segment}\n")

Segment 1:
제1조(목적) 이 규칙은 「도로교통법」 및 동법 시행령에서 위임된 사항과 그 시행에 관하여 필요한 사항을 규정함을 목
적으로 한다.

Segment 2:
제2조(차마에서 제외하는 기구ㆍ장치) ① 「도로교통법」(이하 “법”이라 한다) 제2조제10호에서 “유모차, 보행보조용 의
자차, 노약자용 보행기 등 행정안전부령으로 정하는 기구ㆍ장치”란 너비 1미터 이하인 것으로서 다음 각 호의 기구
ㆍ장치를 말한다. <개정 2023. 10. 19.>
1. 유모차
2. 보행보조용 의자차(「의료기기법」 제19조에 따라 식품의약품안전처장이 정하는 의료기기의 기준규격에 따른 수
동휠체어, 전동휠체어 및 의료용 스쿠터를 말한다)
3. 노약자용 보행기
4. 법 제11조제3항에 따른 놀이기구(어린이가 이용하는 것에 한정한다)
5. 동력이 없는 손수레
6. 이륜자동차, 원동기장치자전거 또는 자전거로서 운전자가 내려서 끌거나 들고 통행하는 것
7. 도로의 보수ㆍ유지, 도로상의 공사 등 작업에 사용되는 기구ㆍ장치(사람이 타거나 화물을 운송하지 않는 것에 한
정한다)
② 법 제2조제17호가목5)에서 “유모차, 보행보조용 의자차, 노약자용 보행기, 제21호의3에 따른 실외이동로봇 등 행
정안전부령으로 정하는 기구ㆍ장치”란 다음 각 호의 기구ㆍ장치를 말한다.<신설 2023. 10. 19.>
1. 제1항 각 호의 어느 하나에 해당하는 기구ㆍ장치
2. 실외이동로봇

제2조의2(자율주행시스템의 종류) 법 제2조제18호의2 후단에 따른 자율주행시스템의 종류에 관하여는 「자동차 및 자
동차부품의 성능과 기준에 관한 규칙」 제111조를 준용한다.


제2조의3(개인형 이동장치의 기준) 법 제2조제19호의2에서 “행정안전부령으로 정하는 것”이란 다음 각 호의 어느 하나
에 해당하는 것으로서 「전기용품 및 생활용품 안전관리법」 제15조제1항에 따라 안전확인의 신고가 된 것을 말한다.
1. 전동킥보드
2. 전동이륜평행차
3. 전동기의 동력만으로 움직일 수 있는 자전거


제2조의4(실외이동로봇

In [3]:
split_segments

['제1조(목적) 이 규칙은 「도로교통법」 및 동법 시행령에서 위임된 사항과 그 시행에 관하여 필요한 사항을 규정함을 목\n적으로 한다.',
 '제2조(차마에서 제외하는 기구ㆍ장치) ① 「도로교통법」(이하 “법”이라 한다) 제2조제10호에서 “유모차, 보행보조용 의\n자차, 노약자용 보행기 등 행정안전부령으로 정하는 기구ㆍ장치”란 너비 1미터 이하인 것으로서 다음 각 호의 기구\nㆍ장치를 말한다. <개정 2023. 10. 19.>\n1. 유모차\n2. 보행보조용 의자차(「의료기기법」 제19조에 따라 식품의약품안전처장이 정하는 의료기기의 기준규격에 따른 수\n동휠체어, 전동휠체어 및 의료용 스쿠터를 말한다)\n3. 노약자용 보행기\n4. 법 제11조제3항에 따른 놀이기구(어린이가 이용하는 것에 한정한다)\n5. 동력이 없는 손수레\n6. 이륜자동차, 원동기장치자전거 또는 자전거로서 운전자가 내려서 끌거나 들고 통행하는 것\n7. 도로의 보수ㆍ유지, 도로상의 공사 등 작업에 사용되는 기구ㆍ장치(사람이 타거나 화물을 운송하지 않는 것에 한\n정한다)\n② 법 제2조제17호가목5)에서 “유모차, 보행보조용 의자차, 노약자용 보행기, 제21호의3에 따른 실외이동로봇 등 행\n정안전부령으로 정하는 기구ㆍ장치”란 다음 각 호의 기구ㆍ장치를 말한다.<신설 2023. 10. 19.>\n1. 제1항 각 호의 어느 하나에 해당하는 기구ㆍ장치\n2. 실외이동로봇\n\n제2조의2(자율주행시스템의 종류) 법 제2조제18호의2 후단에 따른 자율주행시스템의 종류에 관하여는 「자동차 및 자\n동차부품의 성능과 기준에 관한 규칙」 제111조를 준용한다.\n\n\n제2조의3(개인형 이동장치의 기준) 법 제2조제19호의2에서 “행정안전부령으로 정하는 것”이란 다음 각 호의 어느 하나\n에 해당하는 것으로서 「전기용품 및 생활용품 안전관리법」 제15조제1항에 따라 안전확인의 신고가 된 것을 말한다.\n1. 전동킥보드\n2. 전동이륜평행차\n3. 전동기의 동력만으로 움직일 수 있는 자전거\n\n\n

In [4]:
import re
from langchain.document_loaders import PyMuPDFLoader
from sentence_transformers import SentenceTransformer

def clean_legal_text(text):
    """
    Clean legal text by removing unwanted headers, footers, and other repetitive patterns.
    """
    # Remove page headers/footers (e.g., 법제처, 국가법령정보센터, page numbers)
    text = re.sub(r'^\s*법제처.*국가법령정보센터\s*$|^\s*\d+\s*$', '', text, flags=re.MULTILINE)
    
    # Remove contact information and department names
    text = re.sub(r'\(.*?\)\s*\d{2,3}-\d{3,4}-\d{4}', '', text)
    
    # Remove excessive newlines and spaces
    text = re.sub(r'(\n\s*){2,}', '\n', text)  # Collapse multiple newlines
    text = re.sub(r'\s{2,}', ' ', text)        # Collapse multiple spaces
    
    # Remove metadata like [시행 2024.10.20.]
    text = re.sub(r'^\s*\[.*?\]$', '', text, flags=re.MULTILINE)
    
    # Trim leading and trailing whitespace
    return text.strip()

def keyword_based_splitter(text, pattern=r'제\s*\d+\s*조\s*\(.*?\)'):
    """
    Split text based on a keyword pattern like '제 몇 조 (블라 블라)'.
    """
    # Find all matches for the pattern
    matches = re.finditer(pattern, text)

    # Extract start positions of each match
    split_positions = [match.start() for match in matches]

    # Add the end of the text as the last position
    split_positions.append(len(text))

    # Split the text based on the positions
    segments = []
    for i in range(len(split_positions) - 1):
        start = split_positions[i]
        end = split_positions[i + 1]
        segments.append(text[start:end].strip())
    
    return segments

def process_and_embed_pdf(file_path, pattern=r'제\s*\d+\s*조\s*\(.*?\)'):
    """
    Load, clean, split, and embed a PDF file.
    """
    # Load the PDF using PyMuPDFLoader
    loader = PyMuPDFLoader(file_path)
    documents = loader.load()

    # Extract and clean text from the documents
    cleaned_texts = []
    for doc in documents:
        cleaned_texts.append(clean_legal_text(doc.page_content))
    
    # Combine cleaned texts into one and split based on the pattern
    combined_cleaned_text = '\n'.join(cleaned_texts)
    split_segments = keyword_based_splitter(combined_cleaned_text, pattern)
    
    # Load the Jina Embeddings v3 model
    model = SentenceTransformer("jinaai/jina-embeddings-v3", trust_remote_code=True)
    
    # Compute embeddings for the split segments
    embeddings = model.encode(split_segments)
    
    return split_segments, embeddings

# Example usage
file_path = "file/도로 교통법/도로교통법 시행규칙(행정안전부령)(제00431호)(20241020).pdf"  # Replace with your file path
split_segments, embeddings = process_and_embed_pdf(file_path)

print("=== Embeddings Shape ===")
print(len(embeddings), embeddings[0].shape)  # Number of embeddings and dimension of a single embedding

=== Embeddings Shape ===
149 (1024,)


# 2. Chroma 벡터 DB 저장

In [1]:
import re
from langchain.document_loaders import PyMuPDFLoader
from sentence_transformers import SentenceTransformer
from langchain_community.vectorstores import Chroma
from langchain.schema import Document

class SentenceTransformerEmbedding:
    """
    Wrapper for SentenceTransformer to provide embed_documents and embed_query methods.
    """
    def __init__(self, model_name):
        self.model = SentenceTransformer(model_name, trust_remote_code=True)

    def embed_documents(self, texts):
        # Convert embeddings to lists
        return self.model.encode(texts).tolist()

    def embed_query(self, text):
        # Convert embedding to list
        return self.model.encode([text])[0].tolist()


def clean_legal_text(text):
    """
    Clean legal text by removing unwanted headers, footers, and other repetitive patterns.
    """
    text = re.sub(r'^\s*법제처.*국가법령정보센터\s*$|^\s*\d+\s*$', '', text, flags=re.MULTILINE)
    text = re.sub(r'\(.*?\)\s*\d{2,3}-\d{3,4}-\d{4}', '', text)
    text = re.sub(r'(\n\s*){2,}', '\n', text)  # Collapse multiple newlines
    text = re.sub(r'\s{2,}', ' ', text)        # Collapse multiple spaces
    text = re.sub(r'^\s*\[.*?\]$', '', text, flags=re.MULTILINE)
    return text.strip()

def keyword_based_splitter(text, pattern=r'제\s*\d+\s*조\s*\(.*?\)'):
    """
    Split text based on a keyword pattern like '제 몇 조 (블라 블라)'.
    """
    matches = re.finditer(pattern, text)
    split_positions = [match.start() for match in matches]
    split_positions.append(len(text))

    segments = []
    for i in range(len(split_positions) - 1):
        start = split_positions[i]
        end = split_positions[i + 1]
        segments.append(text[start:end].strip())
    
    return segments

def process_and_store_pdf_in_chroma(file_path, pattern=r'제\s*\d+\s*조\s*\(.*?\)', chroma_dir="chroma_data"):
    """
    Process a PDF, compute embeddings, and store them in a Chroma vector database.
    """
    loader = PyMuPDFLoader(file_path)
    documents = loader.load()

    cleaned_texts = [clean_legal_text(doc.page_content) for doc in documents]
    combined_cleaned_text = '\n'.join(cleaned_texts)
    split_segments = keyword_based_splitter(combined_cleaned_text, pattern)
    
    document_objects = [
        Document(page_content=segment, metadata={"source": file_path, "section": f"Section {i+1}"})
        for i, segment in enumerate(split_segments)
    ]
    
    embedding_model = SentenceTransformerEmbedding("jinaai/jina-embeddings-v3")
    
    vectorstore = Chroma.from_documents(
        documents=document_objects,
        embedding=embedding_model,
        persist_directory=chroma_dir
    )
    
    vectorstore.persist()
    return vectorstore

# Example usage
file_path = "file/도로 교통법/도로교통법 시행규칙(행정안전부령)(제00431호)(20241020).pdf"
chroma_dir = "chroma_data"

vectorstore = process_and_store_pdf_in_chroma(file_path, chroma_dir=chroma_dir)

retriever = vectorstore.as_retriever(search_kwargs={'k': 3})
results = retriever.get_relevant_documents("교통안전")

for result in results:
    print("=== Retrieved Document ===")
    print(result.page_content)

/var/folders/2w/p0zgpvps62gb8jlzgy_v7zgh0000gn/T/ipykernel_2001/1433855656.py:74: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()
/var/folders/2w/p0zgpvps62gb8jlzgy_v7zgh0000gn/T/ipykernel_2001/1433855656.py:84: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use invoke instead.
  results = retriever.get_relevant_documents("교통안전")


=== Retrieved Document ===
제141조(교통안전심의위원회의 설치 등) ①도로교통안전에 관한 시책과 교통안전시설 관련 신기술 등을 심의하기 위
하여 경찰청에 교통안전심의위원회(이하 “교통안전위원회”라 한다)를 둔다.
②교통안전위원회는 위원장을 포함하여 25인 이상 30인 이내의 위원으로 구성하되, 위원장은 경찰청 소속 국장급
경찰공무원(자치경찰공무원은 제외한다)으로 하고, 위원은 도로교통안전 관련 분야의 지식과 경험이 풍부한 전문가
또는 공무원 중 경찰청장이 위촉 또는 임명하는 사람이 된다.<개정 2006. 10. 19., 2020. 12. 31.>
③위원 중 공무원이 아닌 위원의 임기는 2년으로 하되, 연임할 수 있다.
④그 밖에 교통안전위원회의 운영 등에 관하여 필요한 사항은 경찰청장이 정한다.
제141조의2(규제의 재검토) 경찰청장은 다음 각 호의 사항에 대하여 해당 호에서 정하는 날을 기준일로 하여 3년마다
(매 3년이 되는 해의 기준일과 같은 날 전까지를 말한다) 그 타당성을 검토하여 개선 등의 조치를 해야 한다. <개정
2014. 12. 31., 2019. 3. 28., 2022. 12. 30.>
1. 제19조에 따른 자동차등의 운행속도: 2014년 1월 1일
1의2. 제23조에 따른 대행법인등의 지정신청 등에 관한 첨부서류: 2022년 1월 1일
1의3. 제33조의3에 따른 개인형 이동장치 승차정원: 2023년 1월 1일
1의4. 제46조제1항 및 별표 16에 따른 운전면허 취소ㆍ정지를 받은 사람에 대한 특별교통안전 의무교육: 2023년
1월 1일
1의5. 제46조의2제1항 및 별표 16에 따른 긴급자동차 운전자에 대한 교통안전교육: 2023년 1월 1일
1의6. 제47조에 따른 교통안전교육기관의 지정신청 등에 관한 첨부서류: 2014년 7월 1일
1의7. 제91조제1항 및 별표 28에 따른 속도위반 운전자에 대한 운전면허 취소ㆍ정지처분 기준: 2023년 1월 1일
2. 제99조에 따른 자동차운전학원의 등록신청에 관한 첨부서류: 

In [7]:
retriever.get_relevant_documents("어린이 보호구역")

[Document(metadata={'section': 'Section 36', 'source': 'file/도로 교통법/도로교통법 시행규칙(행정안전부령)(제00431호)(20241020).pdf'}, page_content='제36조(어린이 보호표지) 영 제31조제2호에 따른 어린이 보호표지는 별표 14와 같다.'),
 Document(metadata={'section': 'Section 13', 'source': 'file/도로 교통법/도로교통법 시행규칙(행정안전부령)(제00431호)(20241020).pdf'}, page_content='제13조(어린이의 보호) ①법 제11조제3항에서 “행정안전부령이 정하는 위험성이 큰 놀이기구”라 함은 다음 각 호의 어\n느 하나에 해당하는 놀이기구를 말한다. <개정 2008. 3. 6., 2013. 3. 23., 2014. 11. 19., 2017. 7. 26.>\n1. 킥보드\n2. 롤러스케이트\n3. 인라인스케이트\n4. 스케이트보드\n5. 그 밖에 제1호 내지 제4호의 놀이기구와 비슷한 놀이기구\n② 법 제11조제3항에서 “행정안전부령이 정하는 인명보호장구”란 제32조제1항제1호 및 제3호부터 제7호까지에 따\n른 기준에 적합한 안전모를 말한다.<개정 2010. 7. 9., 2013. 3. 23., 2014. 11. 19., 2017. 7. 26.>'),
 Document(metadata={'section': 'Section 14', 'source': 'file/도로 교통법/도로교통법 시행규칙(행정안전부령)(제00431호)(20241020).pdf'}, page_content='제14조(어린이집 및 학원의 범위) ① 법 제12조제1항제2호에서 “행정안전부령으로 정하는 어린이집”이란 정원 100명\n이상의 어린이집을 말한다. 다만, 시장등이 관할 경찰서장과 협의하여 어린이집이 소재한 지역의 교통여건 등을 고\n려하여 교통사고의 위험으로부터 어린이를 보호할 필요가 있다고 인정하는 경우에는 정원이 100명 미만의 어린

## 1) 메타 데이터 관리 및 벡터 db 저장 (PDF 하나)

In [1]:
import re
import json
import os
import shutil
from collections import Counter
from langchain.document_loaders import PyMuPDFLoader
from sentence_transformers import SentenceTransformer
from langchain_community.vectorstores import Chroma
from langchain.schema import Document

class SentenceTransformerEmbedding:
    """
    Wrapper for SentenceTransformer to provide embed_documents and embed_query methods.
    """
    def __init__(self, model_name):
        self.model = SentenceTransformer(model_name, trust_remote_code=True)

    def embed_documents(self, texts):
        return self.model.encode(texts).tolist()

    def embed_query(self, text):
        return self.model.encode([text])[0].tolist()

def clean_legal_text(text):
    """
    Clean legal text by removing unwanted headers, footers, and other repetitive patterns.
    """
    text = re.sub(r'^\s*법제처.*국가법령정보센터\s*$|^\s*\d+\s*$', '', text, flags=re.MULTILINE)
    text = re.sub(r'\(.*?\)\s*\d{2,3}-\d{3,4}-\d{4}', '', text)
    text = re.sub(r'(\n\s*){2,}', '\n', text)  # Collapse multiple newlines
    text = re.sub(r'\s{2,}', ' ', text)        # Collapse multiple spaces
    text = re.sub(r'^\s*\[.*?\]$', '', text, flags=re.MULTILINE)
    return text.strip()

def extract_revision_dates(segment):
    """
    Extract revision dates from the text and return as a list of dictionaries with year, month, day.
    """
    revision_pattern = r"(\d{4})\.\s*(\d{1,2})\.\s*(\d{1,2})"
    matches = re.findall(revision_pattern, segment)
    return [{"year": int(year), "month": int(month), "day": int(day)} for year, month, day in matches]

def simplify_related_references(related_refs):
    """
    Simplify related references by keeping only the top-level article (e.g., '제19조'),
    and sort them by frequency (desc) and name (asc) if tied.
    """
    # Simplify to top-level references
    simplified_refs = []
    for ref in related_refs:
        top_level = re.match(r'(제\d+조)', ref)
        if top_level:
            simplified_refs.append(top_level.group(1))
    
    # Count frequencies
    ref_counts = Counter(simplified_refs)
    
    # Sort by frequency (desc), then name (asc)
    sorted_refs = sorted(ref_counts.items(), key=lambda x: (-x[1], x[0]))
    
    # Return only the sorted references
    return [ref for ref, _ in sorted_refs]

def extract_metadata(segment):
    """
    Extract metadata such as article number, title, related references, and revision dates.
    """
    article_match = re.search(r'(제\s*\d+\s*조)\s*\((.*?)\)', segment)
    related_refs = re.findall(r'제\d+조(?:제\d+항)?(?:제\d+호)?', segment)

    # Simplify and sort related references
    simplified_refs = simplify_related_references(related_refs)
    related_refs_str = ", ".join(simplified_refs) if simplified_refs else "Not Applicable"

    revision_dates = extract_revision_dates(segment)
    revision_dates_str = json.dumps(revision_dates) if revision_dates else "Not Applicable"

    metadata = {
        "article": article_match.group(1) if article_match else "Unknown",
        "title": article_match.group(2) if article_match else "Unknown",
        "related_references": related_refs_str,
        "revision_dates": revision_dates_str
    }
    return metadata

def remove_duplicates(documents):
    """
    Remove duplicate documents based on page_content.
    """
    unique_content = {}
    unique_documents = []
    for doc in documents:
        # Check if page_content already exists
        if doc.page_content not in unique_content:
            unique_content[doc.page_content] = doc.metadata
            unique_documents.append(doc)
        else:
            # Update metadata to ensure consistency
            unique_metadata = unique_content[doc.page_content]
            unique_metadata.update(doc.metadata)
            unique_content[doc.page_content] = unique_metadata
    return [
        Document(page_content=content, metadata=meta)
        for content, meta in unique_content.items()
    ]

def keyword_based_splitter(text, pattern=r'제\s*\d+\s*조\s*\(.*?\)'):
    """
    Split text based on a keyword pattern like '제 몇 조 (블라 블라)'.
    """
    matches = re.finditer(pattern, text)
    split_positions = [match.start() for match in matches]
    split_positions.append(len(text))

    segments = []
    for i in range(len(split_positions) - 1):
        start = split_positions[i]
        end = split_positions[i + 1]
        segments.append(text[start:end].strip())
    
    return segments

def reset_chroma_directory(directory):
    """
    Delete the Chroma directory to reset the vector database.
    """
    if os.path.exists(directory):
        shutil.rmtree(directory)

def process_and_store_pdf_in_chroma(file_path, pattern=r'제\s*\d+\s*조\s*\(.*?\)', chroma_dir="chroma_data"):
    """
    Process a PDF, compute embeddings, and store them in a Chroma vector database.
    """
    reset_chroma_directory(chroma_dir)

    loader = PyMuPDFLoader(file_path)
    documents = loader.load()

    # Clean and split text
    cleaned_texts = [clean_legal_text(doc.page_content) for doc in documents]
    combined_cleaned_text = '\n'.join(cleaned_texts)
    split_segments = keyword_based_splitter(combined_cleaned_text, pattern)

    # Create Document objects with metadata
    document_objects = []
    for i, segment in enumerate(split_segments):
        metadata = extract_metadata(segment)
        metadata["source"] = file_path
        document_objects.append(Document(page_content=segment, metadata=metadata))

    # Remove duplicates
    document_objects = remove_duplicates(document_objects)

    # Embed and store in Chroma
    embedding_model = SentenceTransformerEmbedding("jinaai/jina-embeddings-v3")
    vectorstore = Chroma.from_documents(
        documents=document_objects,
        embedding=embedding_model,
        persist_directory=chroma_dir
    )
    vectorstore.persist()
    return vectorstore

# Example usage
file_path = "file/도로 교통법/도로교통법 시행규칙(행정안전부령)(제00431호)(20241020).pdf"
# chroma_dir = "chroma_data"
chroma_dir = "vector_db"

vectorstore = process_and_store_pdf_in_chroma(file_path, chroma_dir=chroma_dir)

retriever = vectorstore.as_retriever(search_kwargs={'k': 5})
retriever.get_relevant_documents("교통안전")

/var/folders/2w/p0zgpvps62gb8jlzgy_v7zgh0000gn/T/ipykernel_2428/2007096168.py:128: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()
/var/folders/2w/p0zgpvps62gb8jlzgy_v7zgh0000gn/T/ipykernel_2428/2007096168.py:138: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use invoke instead.
  retriever.get_relevant_documents("교통안전")


[Document(metadata={'article': '제141조', 'related_references': '', 'revision_dates': '', 'source': 'file/도로 교통법/도로교통법 시행규칙(행정안전부령)(제00431호)(20241020).pdf', 'title': '교통안전심의위원회의 설치 등'}, page_content='제141조(교통안전심의위원회의 설치 등) ①도로교통안전에 관한 시책과 교통안전시설 관련 신기술 등을 심의하기 위\n하여 경찰청에 교통안전심의위원회(이하 “교통안전위원회”라 한다)를 둔다.\n②교통안전위원회는 위원장을 포함하여 25인 이상 30인 이내의 위원으로 구성하되, 위원장은 경찰청 소속 국장급\n경찰공무원(자치경찰공무원은 제외한다)으로 하고, 위원은 도로교통안전 관련 분야의 지식과 경험이 풍부한 전문가\n또는 공무원 중 경찰청장이 위촉 또는 임명하는 사람이 된다.<개정 2006. 10. 19., 2020. 12. 31.>\n③위원 중 공무원이 아닌 위원의 임기는 2년으로 하되, 연임할 수 있다.\n④그 밖에 교통안전위원회의 운영 등에 관하여 필요한 사항은 경찰청장이 정한다.\n제141조의2(규제의 재검토) 경찰청장은 다음 각 호의 사항에 대하여 해당 호에서 정하는 날을 기준일로 하여 3년마다\n(매 3년이 되는 해의 기준일과 같은 날 전까지를 말한다) 그 타당성을 검토하여 개선 등의 조치를 해야 한다. <개정\n2014. 12. 31., 2019. 3. 28., 2022. 12. 30.>\n1. 제19조에 따른 자동차등의 운행속도: 2014년 1월 1일\n1의2. 제23조에 따른 대행법인등의 지정신청 등에 관한 첨부서류: 2022년 1월 1일\n1의3. 제33조의3에 따른 개인형 이동장치 승차정원: 2023년 1월 1일\n1의4. 제46조제1항 및 별표 16에 따른 운전면허 취소ㆍ정지를 받은 사람에 대한 특별교통안전 의무교육: 2023년\n1월 1일\n1의5. 제46조의2제1항 및 별표 16에 따른 긴급자동

In [6]:
retriever.get_relevant_documents("제23조")

[Document(metadata={'article': '제23조', 'related_references': '', 'revision_dates': '', 'source': 'file/도로 교통법/도로교통법 시행규칙(행정안전부령)(제00431호)(20241020).pdf', 'title': '대행법인등의 지정신청 등'}, page_content='제23조(대행법인등의 지정신청 등) ①영 제17조제1항에 따라 대행법인등의 지정을 받으려는 자는 별지 제12호서식의\n대행법인등 지정신청서에 다음 각 호의 서류를 첨부하여 관할경찰서장 또는 시장ㆍ군수ㆍ구청장(자치구가 아닌 구\n의 구청장을 제외한다. 이하 “구청장등”이라 한다)에게 제출해야 한다. 이 경우 전단에 따른 신청서를 받은 관할경찰\n서장 또는 구청장등은 「전자정부법」 제36조제1항에 따른 행정정보의 공동이용을 통하여 법인 등기사항증명서(법인\n인 경우만 해당한다)를 확인해야 한다. <개정 2007. 9. 28., 2009. 11. 27., 2010. 9. 10., 2022. 12. 30.>\n1. 정관(법인인 경우만 해당한다)\n2. 대행업무처리에 관한 업무규정\n3. 삭제<2022. 12. 30.>\n②관할경찰서장 또는 구청장등은 대행법인등을 지정한 때에는 영 제17조제2항에 따라 그 사실을 해당 경찰서 또는\n시ㆍ군ㆍ구의 게시판에 공고하고, 별지 제13호서식에 따른 대행업지정증을 교부하여야 한다.\n③영 제17조제3항에 따라 대행법인등은 5천만원 이상의 손해배상을 위한 이행보증보험 또는 대행법인등에 갈음하\n여 피해자의 손해를 배상하는 보험에 가입하거나 그 소재지를 관할하는 공탁기관에 공탁을 하여야 하며, 경찰서장\n또는 구청장등은 제2항에 따라 대행업지정증을 교부하는 때에 이를 확인하여야 한다.\n④그 밖에 대행법인등의 대행업무수행에 관한 사항으로서 이 규칙에 규정되지 아니한 사항은 시ㆍ도경찰청장이 정\n한다.<개정 2020. 12. 31.>'),
 Document(metadata={'article':

## 2) PDF 폴더 전체 벡터 db 저장

In [1]:
import re
import json
import os
import shutil
import gc
from collections import Counter
from langchain.document_loaders import PyMuPDFLoader
from sentence_transformers import SentenceTransformer
from langchain_community.vectorstores import Chroma
from langchain.schema import Document

class SentenceTransformerEmbedding:
    """
    Wrapper for SentenceTransformer to provide embed_documents and embed_query methods.
    """
    def __init__(self, model_name):
        self.model = SentenceTransformer(model_name, trust_remote_code=True)

    def embed_documents(self, texts):
        return self.model.encode(texts).tolist()

    def embed_query(self, text):
        return self.model.encode([text])[0].tolist()

def clean_legal_text(text):
    """
    Clean legal text by removing unwanted headers, footers, and other repetitive patterns.
    """
    text = re.sub(r'^\s*법제처.*국가법령정보센터\s*$|^\s*\d+\s*$', '', text, flags=re.MULTILINE)
    text = re.sub(r'\(.*?\)\s*\d{2,3}-\d{3,4}-\d{4}', '', text)
    text = re.sub(r'(\n\s*){2,}', '\n', text)  # Collapse multiple newlines
    text = re.sub(r'\s{2,}', ' ', text)        # Collapse multiple spaces
    text = re.sub(r'^\s*\[.*?\]$', '', text, flags=re.MULTILINE)
    return text.strip()

def extract_revision_dates(segment):
    """
    Extract revision dates from the text and return as a list of dictionaries with year, month, day.
    """
    revision_pattern = r"(\d{4})\.\s*(\d{1,2})\.\s*(\d{1,2})"
    matches = re.findall(revision_pattern, segment)
    return [{"year": int(year), "month": int(month), "day": int(day)} for year, month, day in matches]

def simplify_related_references(related_refs):
    """
    Simplify related references by keeping only the top-level article (e.g., '제19조'),
    and sort them by frequency (desc) and name (asc) if tied.
    """
    simplified_refs = []
    for ref in related_refs:
        top_level = re.match(r'(제\d+조)', ref)
        if top_level:
            simplified_refs.append(top_level.group(1))
    
    # Count frequencies
    ref_counts = Counter(simplified_refs)
    
    # Sort by frequency (desc), then name (asc)
    sorted_refs = sorted(ref_counts.items(), key=lambda x: (-x[1], x[0]))
    
    # Return only the sorted references
    return [ref for ref, _ in sorted_refs]

def extract_metadata(segment):
    """
    Extract metadata such as article number, title, related references, and revision dates.
    """
    article_match = re.search(r'(제\s*\d+\s*조)\s*\((.*?)\)', segment)
    related_refs = re.findall(r'제\d+조(?:제\d+항)?(?:제\d+호)?', segment)

    # Simplify and sort related references
    simplified_refs = simplify_related_references(related_refs)
    related_refs_str = ", ".join(simplified_refs) if simplified_refs else "Not Applicable"

    revision_dates = extract_revision_dates(segment)
    revision_dates_str = json.dumps(revision_dates) if revision_dates else "Not Applicable"

    metadata = {
        "article": article_match.group(1) if article_match else "Unknown",
        "title": article_match.group(2) if article_match else "Unknown",
        "related_references": related_refs_str,
        "revision_dates": revision_dates_str
    }
    return metadata

def remove_duplicates(documents):
    """
    Remove duplicate documents based on page_content.
    """
    unique_content = {}
    unique_documents = []
    for doc in documents:
        if doc.page_content not in unique_content:
            unique_content[doc.page_content] = doc.metadata
            unique_documents.append(doc)
        else:
            unique_metadata = unique_content[doc.page_content]
            unique_metadata.update(doc.metadata)
            unique_content[doc.page_content] = unique_metadata
    return [
        Document(page_content=content, metadata=meta)
        for content, meta in unique_content.items()
    ]

def keyword_based_splitter(text, pattern=r'제\s*\d+\s*조\s*\(.*?\)'):
    """
    Split text based on a keyword pattern like '제 몇 조 (블라 블라)'.
    """
    matches = re.finditer(pattern, text)
    split_positions = [match.start() for match in matches]
    split_positions.append(len(text))

    segments = []
    for i in range(len(split_positions) - 1):
        start = split_positions[i]
        end = split_positions[i + 1]
        segments.append(text[start:end].strip())
    
    return segments

def reset_chroma_directory(directory):
    """
    Delete the Chroma directory to reset the vector database.
    """
    if os.path.exists(directory):
        shutil.rmtree(directory)

def process_and_store_pdf_in_chroma(directory_path, pattern=r'제\s*\d+\s*조\s*\(.*?\)', chroma_dir="chroma_data"):
    """
    Process all PDF files in the given directory, compute embeddings, and store them in a Chroma vector database.
    """
    reset_chroma_directory(chroma_dir)

    # Initialize Chroma vectorstore and embedding model
    embedding_model = SentenceTransformerEmbedding("jinaai/jina-embeddings-v3")
    vectorstore = None

    for file_name in os.listdir(directory_path):
        if file_name.endswith(".pdf"):
            file_path = os.path.join(directory_path, file_name)
            print(f"Processing: {file_path}")

            loader = PyMuPDFLoader(file_path)
            documents = loader.load()

            # Process each document's page content separately
            for doc in documents:
                # Clean and split text
                cleaned_text = clean_legal_text(doc.page_content)
                if not cleaned_text.strip():  # Skip if text is empty
                    print(f"Warning: Empty page content in {file_path}, skipping.")
                    continue

                split_segments = keyword_based_splitter(cleaned_text, pattern)

                # Create Document objects with metadata
                document_objects = []
                for segment in split_segments:
                    metadata = extract_metadata(segment)
                    metadata["source"] = file_path
                    document_objects.append(Document(page_content=segment, metadata=metadata))

                # Remove duplicates
                document_objects = remove_duplicates(document_objects)

                # Skip adding if no valid documents
                if not document_objects:
                    print(f"Warning: No valid segments found in {file_path}, skipping.")
                    continue

                # Add documents to Chroma
                if vectorstore is None:
                    vectorstore = Chroma.from_documents(
                        documents=document_objects,
                        embedding=embedding_model,
                        persist_directory=chroma_dir
                    )
                else:
                    vectorstore.add_documents(document_objects)

                # Free memory for the current page content
                del split_segments, document_objects
                gc.collect()

            # Free memory for the current file
            del documents
            gc.collect()

    # Persist vectorstore
    if vectorstore:
        vectorstore.persist()
    return vectorstore

# Example usage
directory_path = "file/도로 교통법"
chroma_dir = "chroma_data_folder"

vectorstore = process_and_store_pdf_in_chroma(directory_path, chroma_dir=chroma_dir)
retriever = vectorstore.as_retriever(search_kwargs={'k': 10})
retriever.get_relevant_documents("교통안전")

Processing: file/도로 교통법/도로교통법(법률)(제20375호)(20250320).pdf
Processing: file/도로 교통법/도로교통법 시행규칙(행정안전부령)(제00507호)(20240731).pdf
Processing: file/도로 교통법/도로교통법(법률)(제20375호)(20240920).pdf
Processing: file/도로 교통법/도로교통법(법률)(제19745호)(20241025).pdf
Processing: file/도로 교통법/도로교통법(법률)(제20155호)(20241025).pdf
Processing: file/도로 교통법/도로교통법(법률)(제20270호)(20240814).pdf
Processing: file/도로 교통법/도로교통법 시행령(대통령령)(제34533호)(20240901).pdf
Processing: file/도로 교통법/도로교통법 시행규칙(행정안전부령)(제00431호)(20241020).pdf


/var/folders/2w/p0zgpvps62gb8jlzgy_v7zgh0000gn/T/ipykernel_3258/3241036885.py:191: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()
/var/folders/2w/p0zgpvps62gb8jlzgy_v7zgh0000gn/T/ipykernel_3258/3241036885.py:200: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use invoke instead.
  retriever.get_relevant_documents("교통안전")


[Document(metadata={'article': '제59조', 'related_references': '제59조', 'revision_dates': 'Not Applicable', 'source': 'file/도로 교통법/도로교통법(법률)(제19745호)(20241025).pdf', 'title': '교통안전시설의 설치 및 관리'}, page_content='제59조(교통안전시설의 설치 및 관리) ① 고속도로의 관리자는 고속도로에서 일어나는 위험을 방지하고 교통의 안전과\n원활한 소통을 확보하기 위하여 교통안전시설을 설치ㆍ관리하여야 한다. 이 경우 고속도로의 관리자가 교통안전시\n설을 설치하려면 경찰청장과 협의하여야 한다.'),
 Document(metadata={'article': '제59조', 'related_references': '제59조', 'revision_dates': 'Not Applicable', 'source': 'file/도로 교통법/도로교통법(법률)(제20155호)(20241025).pdf', 'title': '교통안전시설의 설치 및 관리'}, page_content='제59조(교통안전시설의 설치 및 관리) ① 고속도로의 관리자는 고속도로에서 일어나는 위험을 방지하고 교통의 안전과\n원활한 소통을 확보하기 위하여 교통안전시설을 설치ㆍ관리하여야 한다. 이 경우 고속도로의 관리자가 교통안전시\n설을 설치하려면 경찰청장과 협의하여야 한다.'),
 Document(metadata={'article': '제59조', 'related_references': '제59조', 'revision_dates': 'Not Applicable', 'source': 'file/도로 교통법/도로교통법(법률)(제20375호)(20250320).pdf', 'title': '교통안전시설의 설치 및 관리'}, page_content='제59조(교통안전시설의 설치 및 관리) ① 고속도로의 관리자는 고속도로에서 일어나는 위

In [2]:
retriever.get_relevant_documents("어린이 보호 구역의 제한 속도는 몇인가요?")

[Document(metadata={'article': '제12조', 'related_references': '제2조, 제60조, 제10조, 제12조, 제223조, 제38조, 제55조', 'revision_dates': '[{"year": 2013, "month": 3, "day": 23}, {"year": 2014, "month": 1, "day": 28}, {"year": 2014, "month": 11, "day": 19}, {"year": 2015, "month": 7, "day": 24}, {"year": 2017, "month": 7, "day": 26}, {"year": 2018, "month": 3, "day": 27}, {"year": 2021, "month": 10, "day": 19}, {"year": 2023, "month": 4, "day": 18}, {"year": 2013, "month": 3, "day": 23}, {"year": 2014, "month": 11, "day": 19}, {"year": 2017, "month": 7, "day": 26}, {"year": 2023, "month": 4, "day": 18}, {"year": 2018, "month": 3, "day": 27}]', 'source': 'file/도로 교통법/도로교통법(법률)(제20375호)(20240920).pdf', 'title': '어린이 보호구역의 지정ㆍ해제 및 관리'}, page_content='제12조(어린이 보호구역의 지정ㆍ해제 및 관리) ① 시장등은 교통사고의 위험으로부터 어린이를 보호하기 위하여 필요\n하다고 인정하는 경우에는 다음 각 호의 어느 하나에 해당하는 시설이나 장소의 주변도로 가운데 일정 구간을 어린\n이 보호구역으로 지정하여 자동차등과 노면전차의 통행속도를 시속 30킬로미터 이내로 제한할 수 있다. <개정\n2013. 3. 23., 2014. 1. 28., 2014. 11. 19., 2015. 7. 24., 2017. 7. 26

In [1]:
import re
import json
import os
import shutil
import gc
from collections import Counter
from langchain.document_loaders import PyMuPDFLoader
from sentence_transformers import SentenceTransformer
from langchain_community.vectorstores import Chroma
from langchain.schema import Document

class SentenceTransformerEmbedding:
    """
    Wrapper for SentenceTransformer to provide embed_documents and embed_query methods.
    """
    def __init__(self, model_name):
        self.model = SentenceTransformer(model_name, trust_remote_code=True)

    def embed_documents(self, texts):
        return self.model.encode(texts).tolist()

    def embed_query(self, text):
        return self.model.encode([text])[0].tolist()

def clean_legal_text(text):
    """
    Clean legal text by removing unwanted headers, footers, and other repetitive patterns.
    """
    text = re.sub(r'^\s*법제처.*국가법령정보센터\s*$|^\s*\d+\s*$', '', text, flags=re.MULTILINE)
    text = re.sub(r'\(.*?\)\s*\d{2,3}-\d{3,4}-\d{4}', '', text)
    text = re.sub(r'(\n\s*){2,}', '\n', text)  # Collapse multiple newlines
    text = re.sub(r'\s{2,}', ' ', text)        # Collapse multiple spaces
    text = re.sub(r'^\s*\[.*?\]$', '', text, flags=re.MULTILINE)
    return text.strip()

def extract_revision_dates(segment):
    """
    Extract revision dates from the text and return as a list of dictionaries with year, month, day.
    """
    revision_pattern = r"(\d{4})\.\s*(\d{1,2})\.\s*(\d{1,2})"
    matches = re.findall(revision_pattern, segment)
    return [{"year": int(year), "month": int(month), "day": int(day)} for year, month, day in matches]

def simplify_related_references(related_refs):
    """
    Simplify related references by keeping only the top-level article (e.g., '제19조'),
    and sort them by frequency (desc) and name (asc) if tied.
    """
    simplified_refs = []
    for ref in related_refs:
        top_level = re.match(r'(제\d+조)', ref)
        if top_level:
            simplified_refs.append(top_level.group(1))
    
    # Count frequencies
    ref_counts = Counter(simplified_refs)
    
    # Sort by frequency (desc), then name (asc)
    sorted_refs = sorted(ref_counts.items(), key=lambda x: (-x[1], x[0]))
    
    # Return only the sorted references
    return [ref for ref, _ in sorted_refs]

def extract_metadata(segment):
    """
    Extract metadata such as article number, title, related references, and revision dates.
    """
    article_match = re.search(r'(제\s*\d+\s*조)\s*\((.*?)\)', segment)
    related_refs = re.findall(r'제\d+조(?:제\d+항)?(?:제\d+호)?', segment)

    # Simplify and sort related references
    simplified_refs = simplify_related_references(related_refs)
    related_refs_str = ", ".join(simplified_refs) if simplified_refs else "Not Applicable"

    revision_dates = extract_revision_dates(segment)
    revision_dates_str = json.dumps(revision_dates) if revision_dates else "Not Applicable"

    metadata = {
        "article": article_match.group(1) if article_match else "Unknown",
        "title": article_match.group(2) if article_match else "Unknown",
        "related_references": related_refs_str,
        "revision_dates": revision_dates_str
    }
    return metadata

def remove_duplicates(documents):
    """
    Remove duplicate documents based on page_content.
    """
    unique_content = {}
    unique_documents = []
    for doc in documents:
        if doc.page_content not in unique_content:
            unique_content[doc.page_content] = doc.metadata
            unique_documents.append(doc)
        else:
            unique_metadata = unique_content[doc.page_content]
            unique_metadata.update(doc.metadata)
            unique_content[doc.page_content] = unique_metadata
    return [
        Document(page_content=content, metadata=meta)
        for content, meta in unique_content.items()
    ]

def keyword_based_splitter(text, pattern=r'제\s*\d+\s*조\s*\(.*?\)'):
    """
    Split text based on a keyword pattern like '제 몇 조 (블라 블라)'.
    """
    matches = re.finditer(pattern, text)
    split_positions = [match.start() for match in matches]
    split_positions.append(len(text))

    segments = []
    for i in range(len(split_positions) - 1):
        start = split_positions[i]
        end = split_positions[i + 1]
        segments.append(text[start:end].strip())
    
    return segments

def reset_chroma_directory(directory):
    """
    Delete the Chroma directory to reset the vector database.
    """
    if os.path.exists(directory):
        shutil.rmtree(directory)

def process_and_store_pdf_in_chroma(directory_path, pattern=r'제\s*\d+\s*조\s*\(.*?\)', chroma_dir="chroma_data"):
    """
    Process all PDF files in the given directory, compute embeddings, and store them in a Chroma vector database.
    """
    reset_chroma_directory(chroma_dir)

    # Initialize Chroma vectorstore and embedding model
    embedding_model = SentenceTransformerEmbedding("jinaai/jina-embeddings-v3")
    vectorstore = None

    for file_name in os.listdir(directory_path):
        if file_name.endswith(".pdf"):
            file_path = os.path.join(directory_path, file_name)
            print(f"Processing: {file_path}")

            loader = PyMuPDFLoader(file_path)
            documents = loader.load()

            # Process each document's page content separately
            all_document_objects = []  # To collect all document objects for a single file

            for doc in documents:
                # Clean and split text
                cleaned_text = clean_legal_text(doc.page_content)
                if not cleaned_text.strip():  # Skip if text is empty
                    print(f"Warning: Empty page content in {file_path}, skipping.")
                    continue

                split_segments = keyword_based_splitter(cleaned_text, pattern)

                # Create Document objects with metadata
                document_objects = []
                for segment in split_segments:
                    metadata = extract_metadata(segment)
                    metadata["source"] = file_path
                    document_objects.append(Document(page_content=segment, metadata=metadata))

                # Remove duplicates
                document_objects = remove_duplicates(document_objects)

                # Collect valid document objects
                if document_objects:
                    all_document_objects.extend(document_objects)

                # Free memory for the current page content
                del split_segments, document_objects
                gc.collect()

            # Skip adding if no valid documents for the file
            if not all_document_objects:
                print(f"Warning: No valid segments found in {file_path}, skipping.")
                continue

            # Add documents to Chroma
            if vectorstore is None:
                vectorstore = Chroma.from_documents(
                    documents=all_document_objects,
                    embedding=embedding_model,
                    persist_directory=chroma_dir
                )
            else:
                vectorstore.add_documents(all_document_objects)

            # Free memory for the current file
            del all_document_objects, documents
            gc.collect()

    # Persist vectorstore
    if vectorstore:
        vectorstore.persist()
    return vectorstore

# Example usage
directory_path = "file/도로 교통법"
chroma_dir = "chroma_data_folder_2"

vectorstore = process_and_store_pdf_in_chroma(directory_path, chroma_dir=chroma_dir)
retriever = vectorstore.as_retriever(search_kwargs={'k': 10})
retriever.get_relevant_documents("교통안전")

Processing: file/도로 교통법/도로교통법(법률)(제20375호)(20250320).pdf
Processing: file/도로 교통법/도로교통법 시행규칙(행정안전부령)(제00507호)(20240731).pdf
Processing: file/도로 교통법/도로교통법(법률)(제20375호)(20240920).pdf
Processing: file/도로 교통법/도로교통법(법률)(제19745호)(20241025).pdf
Processing: file/도로 교통법/도로교통법(법률)(제20155호)(20241025).pdf
Processing: file/도로 교통법/도로교통법(법률)(제20270호)(20240814).pdf
Processing: file/도로 교통법/도로교통법 시행령(대통령령)(제34533호)(20240901).pdf
Processing: file/도로 교통법/도로교통법 시행규칙(행정안전부령)(제00431호)(20241020).pdf


/var/folders/2w/p0zgpvps62gb8jlzgy_v7zgh0000gn/T/ipykernel_3347/4262713032.py:197: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()
/var/folders/2w/p0zgpvps62gb8jlzgy_v7zgh0000gn/T/ipykernel_3347/4262713032.py:206: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use invoke instead.
  retriever.get_relevant_documents("교통안전")


[Document(metadata={'article': '제59조', 'related_references': '제59조', 'revision_dates': 'Not Applicable', 'source': 'file/도로 교통법/도로교통법(법률)(제19745호)(20241025).pdf', 'title': '교통안전시설의 설치 및 관리'}, page_content='제59조(교통안전시설의 설치 및 관리) ① 고속도로의 관리자는 고속도로에서 일어나는 위험을 방지하고 교통의 안전과\n원활한 소통을 확보하기 위하여 교통안전시설을 설치ㆍ관리하여야 한다. 이 경우 고속도로의 관리자가 교통안전시\n설을 설치하려면 경찰청장과 협의하여야 한다.'),
 Document(metadata={'article': '제59조', 'related_references': '제59조', 'revision_dates': 'Not Applicable', 'source': 'file/도로 교통법/도로교통법(법률)(제20155호)(20241025).pdf', 'title': '교통안전시설의 설치 및 관리'}, page_content='제59조(교통안전시설의 설치 및 관리) ① 고속도로의 관리자는 고속도로에서 일어나는 위험을 방지하고 교통의 안전과\n원활한 소통을 확보하기 위하여 교통안전시설을 설치ㆍ관리하여야 한다. 이 경우 고속도로의 관리자가 교통안전시\n설을 설치하려면 경찰청장과 협의하여야 한다.'),
 Document(metadata={'article': '제59조', 'related_references': '제59조', 'revision_dates': 'Not Applicable', 'source': 'file/도로 교통법/도로교통법(법률)(제20375호)(20240920).pdf', 'title': '교통안전시설의 설치 및 관리'}, page_content='제59조(교통안전시설의 설치 및 관리) ① 고속도로의 관리자는 고속도로에서 일어나는 위

## 3) 중복 X

In [1]:
import re
import json
import os
import shutil
import gc
from collections import Counter
from langchain.document_loaders import PyMuPDFLoader
from sentence_transformers import SentenceTransformer
from langchain_community.vectorstores import Chroma
from langchain.schema import Document

class SentenceTransformerEmbedding:
    """
    Wrapper for SentenceTransformer to provide embed_documents and embed_query methods.
    """
    def __init__(self, model_name):
        self.model = SentenceTransformer(model_name, trust_remote_code=True)

    def embed_documents(self, texts):
        return self.model.encode(texts).tolist()

    def embed_query(self, text):
        return self.model.encode([text])[0].tolist()

def clean_legal_text(text):
    """
    Clean legal text by removing unwanted headers, footers, and other repetitive patterns.
    """
    text = re.sub(r'^\s*법제처.*국가법령정보센터\s*$|^\s*\d+\s*$', '', text, flags=re.MULTILINE)
    text = re.sub(r'\(.*?\)\s*\d{2,3}-\d{3,4}-\d{4}', '', text)
    text = re.sub(r'(\n\s*){2,}', '\n', text)  # Collapse multiple newlines
    text = re.sub(r'\s{2,}', ' ', text)        # Collapse multiple spaces
    text = re.sub(r'^\s*\[.*?\]$', '', text, flags=re.MULTILINE)
    return text.strip()

def extract_revision_dates(segment):
    """
    Extract revision dates from the text and return as a list of dictionaries with year, month, day.
    """
    revision_pattern = r"(\d{4})\.\s*(\d{1,2})\.\s*(\d{1,2})"
    matches = re.findall(revision_pattern, segment)
    return [{"year": int(year), "month": int(month), "day": int(day)} for year, month, day in matches]

def simplify_related_references(related_refs):
    """
    Simplify related references by keeping only the top-level article (e.g., '제19조'),
    and sort them by frequency (desc) and name (asc) if tied.
    """
    simplified_refs = []
    for ref in related_refs:
        top_level = re.match(r'(제\d+조)', ref)
        if top_level:
            simplified_refs.append(top_level.group(1))
    
    # Count frequencies
    ref_counts = Counter(simplified_refs)
    
    # Sort by frequency (desc), then name (asc)
    sorted_refs = sorted(ref_counts.items(), key=lambda x: (-x[1], x[0]))
    
    # Return only the sorted references
    return [ref for ref, _ in sorted_refs]

def extract_metadata(segment):
    """
    Extract metadata such as article number, title, related references, and revision dates.
    """
    article_match = re.search(r'(제\s*\d+\s*조)\s*\((.*?)\)', segment)
    related_refs = re.findall(r'제\d+조(?:제\d+항)?(?:제\d+호)?', segment)

    # Simplify and sort related references
    simplified_refs = simplify_related_references(related_refs)
    related_refs_str = ", ".join(simplified_refs) if simplified_refs else "Not Applicable"

    revision_dates = extract_revision_dates(segment)
    revision_dates_str = json.dumps(revision_dates) if revision_dates else "Not Applicable"

    metadata = {
        "article": article_match.group(1) if article_match else "Unknown",
        "title": article_match.group(2) if article_match else "Unknown",
        "related_references": related_refs_str,
        "revision_dates": revision_dates_str
    }
    return metadata

def remove_duplicates(documents):
    """
    Remove duplicate documents based on page_content.
    """
    unique_content = {}
    unique_documents = []
    for doc in documents:
        if doc.page_content not in unique_content:
            unique_content[doc.page_content] = doc.metadata
            unique_documents.append(doc)
        else:
            unique_metadata = unique_content[doc.page_content]
            unique_metadata.update(doc.metadata)
            unique_content[doc.page_content] = unique_metadata
    return [
        Document(page_content=content, metadata=meta)
        for content, meta in unique_content.items()
    ]

def keyword_based_splitter(text, pattern=r'제\s*\d+\s*조\s*\(.*?\)'):
    """
    Split text based on a keyword pattern like '제 몇 조 (블라 블라)'.
    """
    matches = re.finditer(pattern, text)
    split_positions = [match.start() for match in matches]
    split_positions.append(len(text))

    segments = []
    for i in range(len(split_positions) - 1):
        start = split_positions[i]
        end = split_positions[i + 1]
        segments.append(text[start:end].strip())
    
    return segments

def reset_chroma_directory(directory):
    """
    Delete the Chroma directory to reset the vector database.
    """
    if os.path.exists(directory):
        shutil.rmtree(directory)

def process_and_store_pdf_in_chroma(directory_path, pattern=r'제\s*\d+\s*조\s*\(.*?\)', chroma_dir="chroma_data"):
    """
    Process all PDF files in the given directory, compute embeddings, and store them in a Chroma vector database.
    """
    reset_chroma_directory(chroma_dir)

    # Initialize Chroma vectorstore and embedding model
    embedding_model = SentenceTransformerEmbedding("jinaai/jina-embeddings-v3")
    vectorstore = None

    for file_name in os.listdir(directory_path):
        if file_name.endswith(".pdf"):
            file_path = os.path.join(directory_path, file_name)
            print(f"Processing: {file_path}")

            loader = PyMuPDFLoader(file_path)
            documents = loader.load()

            # Process each document's page content separately
            all_document_objects = []  # To collect all document objects for a single file

            for doc in documents:
                # Clean and split text
                cleaned_text = clean_legal_text(doc.page_content)
                if not cleaned_text.strip():  # Skip if text is empty
                    print(f"Warning: Empty page content in {file_path}, skipping.")
                    continue

                split_segments = keyword_based_splitter(cleaned_text, pattern)

                # Create Document objects with metadata
                document_objects = []
                for segment in split_segments:
                    metadata = extract_metadata(segment)
                    metadata["source"] = file_path
                    document_objects.append(Document(page_content=segment, metadata=metadata))

                # Remove duplicates
                document_objects = remove_duplicates(document_objects)

                # Collect valid document objects
                if document_objects:
                    all_document_objects.extend(document_objects)

                # Free memory for the current page content
                del split_segments, document_objects
                gc.collect()

            # Skip adding if no valid documents for the file
            if not all_document_objects:
                print(f"Warning: No valid segments found in {file_path}, skipping.")
                continue

            # Add documents to Chroma
            if vectorstore is None:
                vectorstore = Chroma.from_documents(
                    documents=all_document_objects,
                    embedding=embedding_model,
                    persist_directory=chroma_dir
                )
            else:
                vectorstore.add_documents(all_document_objects)

            # Free memory for the current file
            del all_document_objects, documents
            gc.collect()

    # Persist vectorstore
    if vectorstore:
        vectorstore.persist()
    return vectorstore

# Example usage
directory_path = "file/현행법령_copy"
chroma_dir = "chroma_data_folder_final"

vectorstore = process_and_store_pdf_in_chroma(directory_path, chroma_dir=chroma_dir)
retriever = vectorstore.as_retriever(search_kwargs={'k': 10})
retriever.get_relevant_documents("교통안전")

Processing: file/현행법령_copy/차세대전자소송 추진단 설치 및 운영에 관한 규칙(대법원규칙)(제02874호)(20200101).pdf
Processing: file/현행법령_copy/민사조정법(법률)(제16910호)(20200305).pdf
Processing: file/현행법령_copy/취업 후 학자금 상환 특별법 시행규칙(교육부령)(제00333호)(20240701).pdf
Processing: file/현행법령_copy/국군방첩사령부령(대통령령)(제33409호)(20230418).pdf
Processing: file/현행법령_copy/정부대표 및 특별사절의 임명과 권한에 관한 법률(법률)(제17160호)(20200331).pdf
Processing: file/현행법령_copy/달빛철도 건설을 위한 특별법(법률)(제20293호)(20240814).pdf
Processing: file/현행법령_copy/한국마사회법 시행령(대통령령)(제34577호)(20240621).pdf
Processing: file/현행법령_copy/연안관리법 시행규칙(해양수산부령)(제00460호)(20210219).pdf
Processing: file/현행법령_copy/허베이 스피리트호 유류오염사고 피해주민의 지원 및 해양환경의 복원 등에 관한 특별법 시행령(대통령령)(제30977호)(20200828).pdf
Processing: file/현행법령_copy/실내공기질 관리법(법률)(제19720호)(20240315).pdf
Processing: file/현행법령_copy/의료ㆍ요양 등 지역 돌봄의 통합지원에 관한 법률(법률)(제20415호)(20260327).pdf
Processing: file/현행법령_copy/대한민국과아메리카합중국간의상호방위조약제4조에의한시설과구역및대한민국에서의합중국군대의지위에관한협정의시행에관한민사특별법시행령(대통령령)(제27960호)(20170330).pdf
Processing: file/현행법령_copy/핵융합에너지 개발진흥법 시행령(대통령령)(제2

/var/folders/2w/p0zgpvps62gb8jlzgy_v7zgh0000gn/T/ipykernel_1218/2512914261.py:197: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()
/var/folders/2w/p0zgpvps62gb8jlzgy_v7zgh0000gn/T/ipykernel_1218/2512914261.py:206: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use invoke instead.
  retriever.get_relevant_documents("교통안전")


[Document(metadata={'article': '제4조', 'related_references': '제4조', 'revision_dates': '[{"year": 2020, "month": 6, "day": 9}]', 'source': 'file/현행법령_copy/교통안전법(법률)(제19673호)(20240817).pdf', 'title': '교통시설설치ㆍ관리자의 의무'}, page_content='제4조(교통시설설치ㆍ관리자의 의무) 교통시설설치ㆍ관리자는 해당 교통시설을 설치 또는 관리하는 경우 교통안전표\n지 그 밖의 교통안전시설을 확충ㆍ정비하는 등 교통안전을 확보하기 위한 필요한 조치를 강구하여야 한다. <개정\n2020. 6. 9.>'),
 Document(metadata={'article': '제52조', 'related_references': '제52조', 'revision_dates': 'Not Applicable', 'source': 'file/현행법령_copy/교통안전법(법률)(제19673호)(20240817).pdf', 'title': '교통안전정보관리체계의 구축 등'}, page_content='제52조(교통안전정보관리체계의 구축 등) ①교통행정기관의 장은 교통시설ㆍ교통수단 및 교통체계의 안전과 관련된\n제반 교통안전에 관한 정보와 교통사고관련자료등을 통합적으로 유지ㆍ관리할 수 있도록 교통안전정보관리체계를'),
 Document(metadata={'article': '제3조', 'related_references': '제3조', 'revision_dates': '[{"year": 2020, "month": 6, "day": 9}]', 'source': 'file/현행법령_copy/교통안전법(법률)(제19673호)(20240817).pdf', 'title': '국가 등의 의무'}, page_content='제3조(국가 등의 의무) ①국가는 국민의 생명ㆍ신체 및 재산을 보호하기 위하여 교통안전에 관한 종합적인 시책을 수립\n하고 이를 시

In [2]:
result = retriever.get_relevant_documents("어린이 보호 구역의 제한 속도는 몇인가요?")

KeyboardInterrupt: 

# 3. 저장된 벡터 스토어 불러오기

In [ ]:
from langchain_community.vectorstores import Chroma
from sentence_transformers import SentenceTransformer

class SentenceTransformerEmbedding:
    """
    Wrapper for SentenceTransformer to provide embed_documents and embed_query methods.
    """
    def __init__(self, model_name):
        self.model = SentenceTransformer(model_name, trust_remote_code=True)

    def embed_documents(self, texts):
        return self.model.encode(texts).tolist()

    def embed_query(self, text):
        return self.model.encode([text])[0].tolist()

def load_chroma_retriever_with_embedding(chroma_dir, embedding_model_name="jinaai/jina-embeddings-v3", k=10):
    """
    Load Chroma vectorstore from a persisted directory and return a retriever with an embedding function.
    
    Args:
        chroma_dir (str): Path to the Chroma vectorstore directory.
        embedding_model_name (str): Name of the embedding model.
        k (int): Number of relevant documents to retrieve.
    
    Returns:
        retriever: Chroma retriever instance.
    """
    # Initialize the embedding model
    embedding_model = SentenceTransformerEmbedding(embedding_model_name)
    
    # Load Chroma vectorstore with the embedding function
    vectorstore = Chroma(persist_directory=chroma_dir, embedding_function=embedding_model)
    
    # Create retriever
    retriever = vectorstore.as_retriever(search_kwargs={'k': k})
    
    return retriever

# Example usage
chroma_dir = "chroma_data_folder_final"  # Path to the Chroma directory
retriever = load_chroma_retriever_with_embedding(chroma_dir, k=10)

# Test retriever
query = "어린이 보호 구역의 제한 속도를 알려주세요"
relevant_documents = retriever.get_relevant_documents(query)

# Print results
print(f"Query: {query}")
for i, doc in enumerate(relevant_documents, start=1):
    print(f"Document {i}:")
    print(f"Content: {doc.page_content[:200]}...")  # Print the first 200 characters
    print(f"Metadata: {doc.metadata}")
    print("-" * 50)

# 4. Re-ranker

In [5]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from langchain_community.vectorstores import Chroma
from sentence_transformers import SentenceTransformer


# Re-ranker 클래스 정의
class ReRanker:
    def __init__(self, model_name):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
        self.reranker = AutoModelForSequenceClassification.from_pretrained(model_name, trust_remote_code=True)

    def re_rank(self, query, docs):
        inputs = self.tokenizer([(query, doc) for doc in docs], 
                                padding=True, truncation=True, return_tensors='pt')
        with torch.no_grad():
            outputs = self.reranker(**inputs)
            scores = outputs.logits.squeeze(-1)
        ranked_docs = [doc for _, doc in sorted(zip(scores, docs), reverse=True)]
        return ranked_docs


# SentenceTransformer 임베딩 클래스 정의
class SentenceTransformerEmbedding:
    def __init__(self, model_name):
        self.model = SentenceTransformer(model_name, trust_remote_code=True)

    def embed_documents(self, texts):
        return self.model.encode(texts).tolist()

    def embed_query(self, text):
        return self.model.encode([text])[0].tolist()


# Chroma 기반 Retriever 초기화
def load_chroma_retriever_with_embedding(chroma_dir, embedding_model_name="jinaai/jina-embeddings-v3", k=10):
    # Initialize the embedding model
    embedding_model = SentenceTransformerEmbedding(embedding_model_name)
    
    # Load Chroma vectorstore with the embedding function
    vectorstore = Chroma(persist_directory=chroma_dir, embedding_function=embedding_model)
    
    # Create retriever
    retriever = vectorstore.as_retriever(search_kwargs={'k': k})
    
    return retriever


# Chroma 디렉토리 및 모델 설정
chroma_dir = "chroma_data_folder_final"
embedding_model_name = "jinaai/jina-embeddings-v3"
reranker_model_name = "Alibaba-NLP/gte-multilingual-reranker-base"

# Retriever와 Re-ranker 초기화
retriever = load_chroma_retriever_with_embedding(chroma_dir, embedding_model_name, k=10)
reranker = ReRanker(reranker_model_name)

# Query 정의
query = "운전 중 휴대전화를 사용하는 것이 법적으로 금지되어 있나요?"

# Retrieve documents
retrieved_docs = retriever.get_relevant_documents(query)

# Re-rank documents
re_ranked_docs = reranker.re_rank(query, [doc.page_content for doc in retrieved_docs])

# 결과 출력
# print("Re-ranked documents:")
# for idx, doc in enumerate(re_ranked_docs):
#     print(f"{idx+1}. {doc}")

In [6]:
retrieved_docs

[Document(metadata={'article': '제29조', 'related_references': '제29조, 제49조', 'revision_dates': 'Not Applicable', 'source': 'file/현행법령_copy/도로교통법 시행령(대통령령)(제34533호)(20240901).pdf', 'title': '안전운전에 장애를 주지 아니하는 장치'}, page_content='제29조(안전운전에 장애를 주지 아니하는 장치) 법 제49조제1항제10호라목에서 “대통령령으로 정하는 장치”란 손으로\n잡지 아니하고도 휴대용 전화(자동차용 전화를 포함한다)를 사용할 수 있도록 해 주는 장치를 말한다.'),
 Document(metadata={'article': '제8조', 'related_references': '제5조, 제8조', 'revision_dates': 'Not Applicable', 'source': 'file/현행법령_copy/자동차손해배상 보장법(법률)(제20340호)(20240821).pdf', 'title': '운행의 금지'}, page_content='제8조(운행의 금지) 의무보험에 가입되어 있지 아니한 자동차는 도로에서 운행하여서는 아니 된다. 다만, 제5조제4항에\n따라 대통령령으로 정하는 자동차는 운행할 수 있다.'),
 Document(metadata={'article': '제62조', 'related_references': '제62조', 'revision_dates': 'Not Applicable', 'source': 'file/현행법령_copy/도로교통법(법률)(제20375호)(20250320).pdf', 'title': '횡단 등의 금지'}, page_content='제62조(횡단 등의 금지) 자동차의 운전자는 그 차를 운전하여 고속도로등을 횡단하거나 유턴 또는 후진하여서는 아니\n된다. 다만, 긴급자동차 또는 도로의 보수ㆍ유지 등의 작업을 하는 자동차 가운데 고속도로등에서의 위험을 방지ㆍ\n제거하거나 교통사고

In [7]:
re_ranked_docs

['제29조(안전운전에 장애를 주지 아니하는 장치) 법 제49조제1항제10호라목에서 “대통령령으로 정하는 장치”란 손으로\n잡지 아니하고도 휴대용 전화(자동차용 전화를 포함한다)를 사용할 수 있도록 해 주는 장치를 말한다.',
 '제55조(사고발생 시 조치에 대한 방해의 금지) 교통사고가 일어난 경우에는 누구든지 제54조제1항 및 제2항에 따른 운\n전자등의 조치 또는 신고행위를 방해하여서는 아니 된다.',
 '제35조(자동차의 무단 해체ㆍ조작 금지) 누구든지 다음 각 호의 어느 하나에 해당하는 경우를 제외하고는 국토교통부\n령으로 정하는 장치를 자동차에서 해체하거나 조작[자동차의 최고속도를 제한하는 장치 또는 운전자를 지원하는 조\n향장치(이동방향의 결정을 주로 담당하는 조향장치에 추가되어 운전자의 조향을 보조해주는 장치를 말한다. 이하 같\n다)를 조작(造作)하는 경우에 한정한다]하여서는 아니 된다. <개정 2013. 3. 23., 2014. 1. 7., 2017. 12. 26., 2024. 1.\n9.>\n1. 자동차의 점검ㆍ정비 또는 튜닝을 하려는 경우\n2. 폐차하는 경우\n3. 교육ㆍ연구의 목적으로 사용하는 등 국토교통부령으로 정하는 사유에 해당되는 경우',
 '제79조(公開場所에서의 演說ㆍ對談)의 연설ㆍ대담을 위하여 자동차 1대와 휴대용 확성장치 1조를 사용할 수',
 '제8조(운행의 금지) 의무보험에 가입되어 있지 아니한 자동차는 도로에서 운행하여서는 아니 된다. 다만, 제5조제4항에\n따라 대통령령으로 정하는 자동차는 운행할 수 있다.',
 '제62조(횡단 등의 금지) 자동차의 운전자는 그 차를 운전하여 고속도로등을 횡단하거나 유턴 또는 후진하여서는 아니\n된다. 다만, 긴급자동차 또는 도로의 보수ㆍ유지 등의 작업을 하는 자동차 가운데 고속도로등에서의 위험을 방지ㆍ\n제거하거나 교통사고에 대한 응급조치작업을 위한 자동차로서 그 목적을 위하여 반드시 필요한 경우에는 그러하지\n아니하다.',
 '제63조(통행 등의 금지) 자동차(이륜자동차는 긴급자